# Modelisation
## Auteur Linda MARTIN

Aprés la restructuration des données, on peut étudier les différents metaparamètres des modéles utilisables pour effectuer l'apprentissage des données. Comme nos données sont en fonction du temps, elles ne peuvent pas être mélangées. Pour ce type de problème,  les modèles sont de type RNN (réseau neuronal récurrent) car l'apprentissage se fait par séquence de temps et la sortie doit restituer cette séquence. Les 2 modèles que je vais expérimenter sont GRU (Gated Recurrent Unit) et LSTM (Long Short Term Memory) qui drivent de RNN.

In [71]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
import warnings
import pathlib as pl
from typing import Tuple
from typing import List
warnings.filterwarnings("ignore")

In [161]:
class DataLoader:
    def __init__(self, data_path: str="../data/"):
        self.datapath = data_path
        self.train_path = pl.Path(data_path) / "train.csv"
        self.test_path = pl.Path(data_path) / "test.csv"
        self.train_labels_path = pl.Path(data_path) / "train_labels.csv"
        self.target_pairs_path = pl.Path(data_path) / "target_pairs.csv"
        
    def load_data(self) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
        train_df = pd.read_csv(self.train_path, index_col='date_id')
        test_df = pd.read_csv(self.test_path, index_col='date_id')
        train_labels_df = pd.read_csv(self.train_labels_path, index_col='date_id')
        target_pairs_df = pd.read_csv(self.target_pairs_path)
        return train_df, test_df, train_labels_df, target_pairs_df

class PreProcess:
    def __init__(self):
        pass

    def get_train_info(self, df):
        """ Construction d'une data des entêtes de colonnes.
        Args:
            df (pd.DataFrame): Input dataframe d'entrainement.
        Returns:
            pd.DataFrame: Détail des informations de chaque colonne.
        """
        df_names = df.columns
        # Fonction pour nettoyer et split les noms
        def clean_and_split(name):
            name = name.replace("open_interest", "open interest")
            name = name.replace("settlement_price", "settlement price")
            name = name.replace("US_Stock", "US Stock")
            name = name.replace("adj_close", "Close")
            name = name.replace("adj_", "adjusted ")
            name = name.replace("-", "_")
            return name.split("_")

        # Création du DataFrame d'infos
        df_info = pd.DataFrame(
            {
            'Column': df_names,
            'Split': [clean_and_split(name) for name in df_names]
        })

        df_info['Category'] = df_info['Split'].apply(lambda x: x[0])    
        df_info['Ticker'] = df_info['Split'].apply(
        lambda x: "_".join(x[1:-1]) if len(x) > 2 else x[-1] if len(x) == 2 else ""
        )
        df_info['Type'] = df_info['Split'].apply(lambda x: x[-1])

        # Nettoyage final
        df_info['Ticker'] = df_info.apply(
            lambda row: row['Type'] if row['Ticker'] == "" else row['Ticker'], axis=1
        )
        df_info['Column_Id'] = df_info.index + 1

        # Sélection des colonnes finales
        df_info = df_info[['Column_Id', 'Column', 'Category', 'Ticker', 'Type']]    
        return df_info
    
    def get_preprocess_data(self, df, cond):
        # Fonction pour obtenir les données prétraitées en fonction de la condition
        if cond.Column.size > 0:
            return df[cond.Column.values[0]]
        else:
            return None
        
    def preprocess(self, df: pd.DataFrame) -> pd.DataFrame:
        df_processed = df.copy()
        df_processed = df_processed.drop(columns=['is_score'], errors='ignore')
        df_info = self.get_train_info(df_processed)


        df_processed.reset_index(inplace=True)
        # On renomme la date_id en date
        df_processed = df_processed.rename({'date_id': 'date'}, axis='columns')
        # Initialisation du DataFrame résultat
        result = pd.DataFrame(columns=['date', 'id', 'close', 'open', 'high', 'low', 'volume', 'sprice', 'interest'])
        
        for  Category  in df_info.groupby('Category').groups.keys():
            txtCategory=Category.replace(' ','_')
            for label in df_info[(df_info.Category==Category)].groupby('Ticker').groups.keys():
                temp_df = pd.DataFrame()
                temp_df['date'] = df_processed['date']
                temp_df['id'] = f'{txtCategory}_{label}'

                if Category in ['FX','LME']:
                    temp_df['close'] = df_processed[df_info[(df_info.Category==Category) & (df_info.Ticker==label)].Column.values[0]]
                    temp_df['open'] = None
                    temp_df['high'] = None
                    temp_df['low'] = None
                    temp_df['volume'] = None
                    temp_df['sprice'] = None
                    temp_df['interest'] = None
                else:
                    temp_df['close'] = self.get_preprocess_data(df_processed,df_info[(df_info.Category==Category) & (df_info.Ticker==label) & (df_info.Type.isin(['Close', 'adjusted close']))])
                    temp_df['open'] = self.get_preprocess_data(df_processed,df_info[(df_info.Category==Category) & (df_info.Ticker==label) & (df_info.Type.isin(['Open','adjusted open']))])
                    temp_df['high'] = self.get_preprocess_data(df_processed,df_info[(df_info.Category==Category) & (df_info.Ticker==label) & (df_info.Type.isin(['High','adjusted high']))])
                    temp_df['low'] = self.get_preprocess_data(df_processed,df_info[(df_info.Category==Category) & (df_info.Ticker==label) & (df_info.Type.isin(['Low','adjusted low']))])
                    temp_df['volume'] = self.get_preprocess_data(df_processed,df_info[(df_info.Category==Category) & (df_info.Ticker==label) & (df_info.Type.isin(['Volume', 'adjusted volume']))])
                    temp_df['sprice'] = self.get_preprocess_data(df_processed,df_info[(df_info.Category==Category) & (df_info.Ticker==label) & (df_info.Type.isin(['settlement price','adjusted settlement price']))])
                    temp_df['interest'] = self.get_preprocess_data(df_processed,df_info[(df_info.Category==Category) & (df_info.Ticker==label) & (df_info.Type.isin(['open interest','adjusted open interest']))])
                result = pd.concat([result, temp_df], ignore_index=True)
        
        # Réinitialiser l'index
        result = result.reset_index(drop=True)
        
        # Trier par date et id 
        result = result.sort_values(['date', 'id']).reset_index(drop=True)
        
        return result  
    
class FeatureEngineer:
    """Class to handle feature engineering"""
    def __init__(self):
        pass

    def add_lag_features(self,
        df: pd.DataFrame, 
        lags: List[int], 
        date_col: str = 'date'
        ) -> pd.DataFrame:
        """
        Add lag features for specified columns and lags.
        """
        df = df.sort_values(date_col)
        cols = set(df.columns)
        cols.remove('id')
        cols.remove('date')
        for col in cols:
            for lag in lags:
                df[f'{col}_lag{lag}'] = df.groupby('id')[col].shift(lag)
        return df
    
    def add_rolling_features(self,
        df: pd.DataFrame,
        windows: List[int],
        date_col: str = 'date') -> pd.DataFrame:
        """ 
        Add rolling mean and std features for specified columns and windows.
        """
        df = df.sort_values(date_col)
        cols = set(df.columns)
        cols.remove('id')
        cols.remove('date')
        for col in cols:
            for window in windows:
                df[f'{col}_rollmean{window}'] = df.groupby('id')[col].transform(lambda x: x.rolling(window, min_periods=1).mean())
                df[f'{col}_rollstd{window}'] = df.groupby('id')[col].transform(lambda x: x.rolling(window, min_periods=1).std())
                df[f'{col}_rollmin{window}'] = df.groupby('id')[col].transform(lambda x: x.rolling(window, min_periods=1).min())
                df[f'{col}_rollmax{window}'] = df.groupby('id')[col].transform(lambda x: x.rolling(window, min_periods=1).max())
        return df
    
    def prepare_features(self, train_df, test_df):
        """Engineer features for training and testing data"""
        try:
            # Combine train and test for feature engineering
            train_cols = set(train_df.columns)
            test_cols = set(test_df.columns)
            common_cols = train_cols.intersection(test_cols)
            combined_data = pd.concat([
                train_df[list(common_cols)],
                test_df[list(common_cols)]
            ], axis=0).reset_index(drop=True)
            
            combined_data = train_df.merge(
                test_df,
                on='date',
                how='inner',   # ou 'left'
                suffixes=('_t2', '_t1')
            )

            combined_data = self.add_lag_features(combined_data, lags=[1, 2, 3, 5, 7])
            
            # Add rolling features
            combined_data = self.add_rolling_features(combined_data, windows=[5, 10, 15])
            # Handle missing values
            combined_data = combined_data.fillna(method='ffill').fillna(method='bfill').fillna(0)

            # Split back to train and test
            train_size = len(train_df)
            train_features = combined_data.iloc[:train_size]
            test_features = combined_data.iloc[train_size:]

            return train_features, test_features
        except Exception as e:
            print(f"Feature preparation failed: {e}")
            raise

In [162]:
    
class FeatureTarget:
    """Class to handle target feature engineering"""
    def __init__(self):
        pass
    
    def prepare_targets(self, train_labels_df: pd.DataFrame) -> pd.DataFrame:
        """Prepare target information from pairs DataFrame.
        Args:
            pairs (pd.DataFrame): DataFrame containing 'pair' column.   
        Returns:
            pd.DataFrame: DataFrame with target information including price_1, price_2, and is_pair.
        """
        target_cols = [col for col in train_labels_df.columns if col not in ['timestamp', 'id']]
        display(target_cols)
        target_values = train_labels_df[target_cols].values
        return target_values, target_cols
    
    def prepare_targets_info(self, pairs: pd.DataFrame) -> pd.DataFrame:
        """Prepare target information from pairs DataFrame.
        Args:
            pairs (pd.DataFrame): DataFrame containing 'pair' column.
        Returns:
            pd.DataFrame: DataFrame with target information including price_1, price_2, and is_pair.
        """

        target_definitions = pairs["pair"].str.split(" - ", expand=True)
        target_info = pairs.copy()

        # Colonnes price_1 et price_2 (équivalent aux colonnes [,1] et [,2])
        target_info["price_1"] = target_definitions[0]
        target_info["price_2"] = target_definitions[1]

        # is.pair = second élément non vide
        target_info['is_pair'] = target_info['price_2'].apply(lambda x:x is not None)

        # Retirer la colonne "pair"
        target_info = target_info.drop(columns=["pair"])
        return target_info

In [163]:
dataLoader = DataLoader()
featureTarget = FeatureTarget()
featureEngineer = FeatureEngineer()
preProcess = PreProcess()

In [164]:
train_df, test_df, train_labels_df, target_pairs_df = dataLoader.load_data()
train_df_process = preProcess.preprocess(train_df)
test_df_process = preProcess.preprocess(test_df)



In [171]:
train_cols = set(train_df.columns)
test_cols = set(test_df.columns)
common_cols = train_cols.intersection(test_cols)
combined_data = pd.concat([
                train_df[list(common_cols)],
                test_df[list(common_cols)]
            ], axis=0).reset_index(drop=True)
display(len(combined_data))
train_size = len(train_df)
train_features = combined_data.iloc[:train_size]
display(len(train_features))


2007

1917

In [166]:
target_values,target_cols = featureTarget.prepare_targets(train_labels_df)
display(target_values)


['target_0',
 'target_1',
 'target_2',
 'target_3',
 'target_4',
 'target_5',
 'target_6',
 'target_7',
 'target_8',
 'target_9',
 'target_10',
 'target_11',
 'target_12',
 'target_13',
 'target_14',
 'target_15',
 'target_16',
 'target_17',
 'target_18',
 'target_19',
 'target_20',
 'target_21',
 'target_22',
 'target_23',
 'target_24',
 'target_25',
 'target_26',
 'target_27',
 'target_28',
 'target_29',
 'target_30',
 'target_31',
 'target_32',
 'target_33',
 'target_34',
 'target_35',
 'target_36',
 'target_37',
 'target_38',
 'target_39',
 'target_40',
 'target_41',
 'target_42',
 'target_43',
 'target_44',
 'target_45',
 'target_46',
 'target_47',
 'target_48',
 'target_49',
 'target_50',
 'target_51',
 'target_52',
 'target_53',
 'target_54',
 'target_55',
 'target_56',
 'target_57',
 'target_58',
 'target_59',
 'target_60',
 'target_61',
 'target_62',
 'target_63',
 'target_64',
 'target_65',
 'target_66',
 'target_67',
 'target_68',
 'target_69',
 'target_70',
 'target_71',
 '

array([[ 0.0059485 , -0.00285132, -0.0046752 , ...,  0.03823403,
                nan,  0.02730989],
       [ 0.00578281, -0.02411763, -0.00705199, ...,  0.02502068,
         0.00354846,  0.02094043],
       [ 0.00104825,  0.02383639, -0.00893406, ...,  0.00483537,
        -0.00907498,  0.00170587],
       ...,
       [-0.00229414,  0.01289763,  0.00997804, ..., -0.01330409,
        -0.00552677, -0.12768791],
       [        nan,         nan,         nan, ..., -0.00692833,
         0.00680538, -0.01218726],
       [        nan,         nan,         nan, ...,         nan,
         0.01256203,         nan]], shape=(1917, 424))

In [167]:
train_features, test_features = featureEngineer.prepare_features(train_df_process,test_df_process)


Feature preparation failed: 'id'


KeyError: 'id'

In [152]:
display(train_features)
target_info = featureTarget.prepare_targets_info(target_pairs_df)
display(target_info)

,interest,id,high,open,low,volume,sprice,date,close,interest_lag1,...,open_lag5_rollmin5,open_lag5_rollmax5,open_lag5_rollmean10,open_lag5_rollstd10,open_lag5_rollmin10,open_lag5_rollmax10,open_lag5_rollmean15,open_lag5_rollstd15,open_lag5_rollmin15,open_lag5_rollmax15
0,1768.0,FX_AUDCAD,24.5242,23.9998,23.8900,2464147.0,4730.0,0,0.979601,1768.0,...,147.0081,147.0081,147.00810,0.274145,147.0081,147.0081,147.008100,0.274145,147.0081,147.0081
2,1768.0,FX_AUDJPY,24.5242,23.9998,23.8900,2464147.0,4730.0,0,87.933498,1768.0,...,147.0081,147.0081,147.00810,0.274145,147.0081,147.0081,147.008100,0.274145,147.0081,147.0081
3,1768.0,FX_AUDNZD,24.5242,23.9998,23.8900,2464147.0,4730.0,0,1.103011,1768.0,...,147.0081,147.0081,147.00810,0.274145,147.0081,147.0081,147.008100,0.274145,147.0081,147.0081
4,1768.0,FX_AUDUSD,24.5242,23.9998,23.8900,2464147.0,4730.0,0,0.783393,1768.0,...,147.0081,147.0081,147.00810,0.274145,147.0081,147.0081,147.008100,0.274145,147.0081,147.0081
5,1768.0,FX_CADCHF,24.5242,23.9998,23.8900,2464147.0,4730.0,0,0.776874,1768.0,...,147.0081,147.0081,147.00810,0.274145,147.0081,147.0081,147.008100,0.274145,147.0081,147.0081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280537,9665.0,US_Stock_EMB,90.8473,90.7680,90.3765,8001299.0,14730.0,1871,90.440900,9158.0,...,89.6381,90.0048,89.89879,0.169768,89.6381,90.1139,89.976427,0.180164,89.6381,90.1733
280538,9665.0,US_Stock_ENB,43.5751,42.9447,42.8167,1884488.0,14730.0,1871,43.565300,9158.0,...,41.9499,42.8758,42.10457,0.350060,41.6643,42.8758,41.878680,0.490887,40.9108,42.8758
280539,9665.0,US_Stock_EOG,124.3516,122.8256,121.8891,4074876.0,14730.0,1871,123.172400,9158.0,...,119.9024,123.5737,122.43964,1.710212,119.9024,125.7885,122.961360,1.998596,119.9024,125.7885
280540,9665.0,US_Stock_EWJ,71.3400,70.7700,70.7600,2640384.0,14730.0,1871,71.320000,9158.0,...,69.3700,70.6700,69.74700,0.737052,68.4800,70.6700,69.608667,0.704210,68.4800,70.6700


,target,lag,price_1,price_2,is_pair
0,target_0,1,US_Stock_VT_adj_close,None,False
1,target_1,1,LME_PB_Close,US_Stock_VT_adj_close,True
2,target_2,1,LME_CA_Close,LME_ZS_Close,True
3,target_3,1,LME_AH_Close,LME_ZS_Close,True
4,target_4,1,LME_AH_Close,JPX_Gold_Standard_Futures_Close,True
...,...,...,...,...,...
419,target_419,4,FX_NOKUSD,LME_AH_Close,True
420,target_420,4,JPX_Gold_Standard_Futures_Close,US_Stock_RY_adj_close,True
421,target_421,4,US_Stock_EWT_adj_close,LME_AH_Close,True
422,target_422,4,JPX_Platinum_Standard_Futures_Close,FX_NOKCHF,True


In [178]:
fx_nokusd_data = train_features[train_features['id'] == 'US_Stock_CCJ']
display(fx_nokusd_data)

KeyError: 'id'

In [99]:
### import libraries
import torch
import torch.nn as nn
import numpy as np

class LSTMModel(nn.Module):
  """LSTM model for time series prediction"""
  def __init__(self,input_size,hidden_size=128,num_layers=2, dropout=0.2):
    super().__init__()

    # store parameters
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.num_layers = num_layers

    # RNN Layer (notation: LSTM \in RNN)
    self.lstm = nn.LSTM(
      input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0,
            batch_first=True
            )
    
    self.dropout = nn.Dropout(dropout)
    # linear layer for output
    self.fc = nn.Linear(hidden_size, 1)
    
  def forward(self,x):
    lstm_out, _ = self.lstm(x)
    last_output = lstm_out[:, -1, :]
    out = self.dropout(last_output)
    out = self.fc(out)
    return out
  

class GRUModel(nn.Module):
    """GRU model for time series prediction"""
    def __init__(self, input_size, hidden_size=128, num_layers=2, dropout=0.2):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.gru = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0,
            batch_first=True
        )
        
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        gru_out, _ = self.gru(x)
        last_output = gru_out[:, -1, :]
        out = self.dropout(last_output)
        out = self.fc(out)
        return out

In [158]:
# train target values
def convert_to_colname(colname:str)-> str:
    if colname is None:
        return None
    if colname.startswith("FX_"):
        return colname
    colname = colname.replace("_adj","")
    colname = colname.rsplit('_', 1)[0]
    return colname

indexe_target:int = 0
target_name = target_info.loc[indexe_target,'target']
target_col1 = convert_to_colname(target_info.loc[indexe_target,'price_1'])

display(target_col1)  
target_col2 = convert_to_colname(target_info.loc[indexe_target,'price_2'])

display(target_col2)  

def train_features_build(target_col1,target_col2):
 # Filter for the two targets
    if target_col2 is not None:
        df_target2 = train_features[train_features['id'] == target_col2]
        df_target1 = train_features[train_features['id'] == target_col1]
        
        # Concatenate the two DataFrames side by side (axis=1)
        result = df_target2.merge(
    df_target1,
    on='date',
    how='inner',   # ou 'left'
    suffixes=('_t2', '_t1')
)
        result = result.drop(['id', 'date'], axis=1, errors='ignore')
        
        return result
    else:
    
        df_target1 = train_features[train_features['id'] == target_col1]
        # Concatenate the two DataFrames side by side (axis=1)
        result = df_target1
        result = result.drop(['id', 'date'], axis=1, errors='ignore')
        
        return result

display(train_features_build(target_col1,target_col2)) 

'US_Stock_VT'

None

,interest,high,open,low,volume,sprice,close,interest_lag1,interest_lag2,interest_lag3,...,open_lag5_rollmin5,open_lag5_rollmax5,open_lag5_rollmean10,open_lag5_rollstd10,open_lag5_rollmin10,open_lag5_rollmax10,open_lag5_rollmean15,open_lag5_rollstd15,open_lag5_rollmin15,open_lag5_rollmax15
131,1768.0,63.9271,63.6198,63.5857,1267705.0,4730.0,63.9271,1768.0,128380.0,1768.0,...,147.0081,147.0081,147.00810,0.274145,147.0081,147.0081,147.008100,0.274145,147.0081,147.0081
274,1768.0,64.3793,64.0039,64.0039,768549.0,4730.0,64.3623,1768.0,128380.0,1768.0,...,147.0081,147.0081,147.00810,0.274145,147.0081,147.0081,147.008100,0.274145,147.0081,147.0081
417,1323.0,64.8316,64.6610,64.6439,819369.0,3423.0,64.7463,1768.0,128380.0,1768.0,...,147.0081,147.0081,147.00810,0.274145,147.0081,147.0081,147.008100,0.274145,147.0081,147.0081
560,8635.0,65.1389,64.9341,64.8231,925118.0,204.8,65.1218,7391.0,128380.0,1768.0,...,147.0081,147.0081,147.00810,0.274145,147.0081,147.0081,147.008100,0.274145,147.0081,147.0081
703,8635.0,65.2071,65.0621,64.9981,723160.0,204.8,65.1901,2026.0,1768.0,1768.0,...,147.0081,147.0081,147.00810,0.274145,147.0081,147.0081,147.008100,0.274145,147.0081,147.0081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280310,45881.0,117.9944,117.9047,117.0176,1690125.0,14300.0,117.4063,46412.0,46412.0,47065.0,...,115.7118,116.4594,116.64577,0.878431,115.7118,118.0342,117.307947,1.205360,115.7118,118.8117
267398,2081.0,117.9944,117.9047,117.0176,1690125.0,346.9,117.4063,2081.0,2122.0,2122.0,...,115.7118,116.4594,116.41353,0.771000,115.7118,118.0342,117.141153,1.243569,115.7118,118.8117
267541,9665.0,119.0492,117.6455,117.3764,1170651.0,14631.0,118.3731,9665.0,9158.0,9158.0,...,115.7118,117.1372,116.42051,0.777863,115.7118,118.0342,117.039487,1.170527,115.7118,118.8117
280454,3490.0,119.0492,117.6455,117.3764,1170651.0,14730.0,118.3731,3490.0,3311.0,3311.0,...,115.7118,117.1372,116.19724,0.550386,115.7118,117.1372,116.838807,1.100962,115.7118,118.8117


In [159]:
from torch.utils.data import Dataset, DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class TimeSeriesDataset(Dataset):
    """Custom Dataset for time series data"""
    def __init__(self, X, y, sequence_length=10):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)
        self.sequence_length = sequence_length
        
    def __len__(self):
        return len(self.X) - self.sequence_length
        
    def __getitem__(self, idx):
        return (
            self.X[idx:idx + self.sequence_length],
            self.y[idx + self.sequence_length]
        )
    
def prepare_data(self, X, y, sequence_length=10, batch_size=32):
        """Prepare data for training"""
        # Scale features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        # Create dataset and dataloader
        dataset = TimeSeriesDataset(X_scaled, y, sequence_length)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
        
        return dataloader, scaler
        

def train_models_for_target(train_features:pd.DataFrame,
                target_values:np.ndarray,
                target_col1:str,
                target_col2:str,
                model_type:str='LSTM',
                num_epochs:int=10,
                batch_size:int=32,
                learning_rate:float=0.001):
    
    valid_indices = ~np.isnan(target_values)
    X_valid = train_features_build(target_col1,target_col2)[valid_indices]
    y_valid = target_values[valid_indices]


    # Simple train/validation split (last 20% for validation)
    split_idx = int(len(X_valid) * 0.8)
    X_train = X_valid[:split_idx]
    y_train = y_valid[:split_idx]
    
    X_val = X_valid[split_idx:]
    y_val = y_valid[split_idx:]
    # Prepare data
    train_loader, scaler = prepare_data(
                    X_train, y_train, 
                    10, batch_size
                )
    
    # Initialize model
    input_size = X_train.shape[1]
    if model_type == 'LSTM':
        model = LSTMModel(input_size)
    elif model_type == 'GRU':
        model = GRUModel(input_size)
    else:
        raise ValueError("Invalid model type. Choose 'LSTM' or 'GRU'.")
    
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    train_losses = []
    val_losses = []
    # Training loop
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs.squeeze(), batch_y)
                loss.backward()
                optimizer.step()
                
                train_loss += loss.item()
        train_loss /= len(train_loader)
        train_losses.append(train_loss)
        
        loss.backward()
        optimizer.step()
        
        if (epoch+1) % 1 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    
    return model


In [160]:
num_targets = min(1, len(target_cols))
selected_targets = target_cols[:num_targets]
for target_idx, target_name in enumerate(selected_targets):
    print(f"\nTarget {target_idx + 1}/{num_targets}: {target_name}")
    print(f"\nTarget {target_info.loc[target_idx,'target']}")
    # Get target values
    price1 = convert_to_colname(target_info.loc[target_idx,'price_1'])
    price2 = convert_to_colname(target_info.loc[target_idx,'price_2'])
    y = target_values[:, target_idx]
    # Train models for this target
    target_models = train_models_for_target(target_name, y, price1, price2)


Target 1/1: target_0

Target target_0


ValueError: Item wrong length 1917 instead of 1916.